<a href="https://colab.research.google.com/github/WZHOU007-0912/pages/blob/master/Pytorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
import time, json, csv
from glob import glob

import numpy as np
import pandas as pd

# %% deep learning related 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

#   progress bar and pretty print
from tqdm import tqdm
from pprint import pprint

# upload the data from local drive
from google.colab import files
import io
import torchvision.transforms as transforms
from sklearn.utils import shuffle
from torch.autograd import Variable
import re

## 1. 資料前處理 - Data Preprocessing
首先，我們需要將我們的資料整理成 model 可以處理的形式

### 1.1 資料讀取

In [ ]:
#files.upload()
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

### 1.2 資料處理操作

In [ ]:
test_passenger = test_data['PassengerId']

In [ ]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## 2. Model Construction

In [ ]:
def preprocess (data):

  data['Sex'].replace('female', 1, inplace = True)
  data['Sex'].replace('male', 0, inplace = True)

  data['Age'].fillna(data['Age'].median(), inplace = True)

  drop_column = ['Ticket','Cabin', 'PassengerId']
  data.drop(drop_column, axis = 1, inplace = True)

  data['Fare'].fillna(data['Fare'].median(), inplace = True)

  data['Embarked'].replace('S', 1, inplace = True)
  data['Embarked'].replace('C', 2, inplace = True)
  data['Embarked'].replace('Q', 3, inplace = True)
  data['Embarked'].fillna(0, inplace = True)

  #data = data.astype(np.float32)

  return(data)

In [ ]:
train = preprocess(train_data)
test = preprocess(test_data)

In [ ]:
all_data = [train, test]
for i in all_data:
  i['FamilySize'] = i['SibSp'] + i['Parch'] + 1

def get_title(name):
  title_search = re.search('([A-Za-z]+)\.', name)
  if title_search:
    return title_search.group(1)
  else:
    return ''

for i in all_data:
  i['Title'] = i['Name'].apply(get_title)

for i in all_data:
  i['Title'] = i['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don','Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  i['Title'] = i['Title'].replace('Mlle', 'Miss')
  i['Title'] = i['Title'].replace('Ms', 'Miss')
  i['Title'] = i['Title'].replace('Mme', 'Mrs')

for i in all_data:
  i['Age_bin'] = pd.cut(i['Age'], bins = [0,12,20,40,120], labels=['Children','Teenage','Adult','Elder'])

for i in all_data:
  i['Fare_bin'] = pd.cut(i['Fare'], bins= [0,7.91,14.45,31,120], labels = ['Low', 'Median', 'Average', 'High'])

for i in all_data:
  drop_column = ['Age', 'Fare', 'Name']
  i.drop(drop_column, axis = 1, inplace = True)


In [ ]:
train.head(3)

,Survived,Pclass,Sex,SibSp,Parch,Embarked,FamilySize,Title,Age_bin,Fare_bin
0,0,3,0,1,0,1.0,2,Mr,Adult,Low
1,1,1,1,1,0,2.0,2,Mrs,Adult,High
2,1,3,1,0,0,1.0,1,Miss,Adult,Median


In [ ]:
test.head(3)

,Pclass,Sex,SibSp,Parch,Embarked,FamilySize,Title,Age_bin,Fare_bin
0,3,0,0,0,3,1,Mr,Adult,Low
1,3,1,1,0,1,2,Mrs,Elder,Low
2,2,0,0,0,3,1,Mr,Elder,Median


In [ ]:
train = pd.get_dummies(train, columns = ['Embarked', 'Title', 'Age_bin', 'Fare_bin'], prefix = ['Em_Type', 'Title', 'Age_Type', 'Fare_Type'])

In [ ]:
test = pd.get_dummies(test, columns = ['Embarked', 'Title', 'Age_bin', 'Fare_bin'], prefix = ['Em_Type', 'Title', 'Age_Type', 'Fare_Type'])


In [ ]:
x_test = test.iloc[:,:].values

x_test.shape

(418, 21)

In [ ]:
drop_column = ['Em_Type_0.0']
train.drop(drop_column, axis = 1, inplace = True)

In [ ]:
x_train = train.iloc[:,1:].values
y_train = train.iloc[:,0].values
x_train.shape

(891, 21)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(21, 512)
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x
model = Net()

## 3. Training

In [ ]:
batch_size = 64
batch_num = len(x_train) // batch_size
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epoch = 600
train_loss = 0

for epoch in range(num_epoch):
    #model.train()
    x_train, y_train = shuffle(x_train, y_train)
    for i in range(batch_num):
      start = i * batch_size
      end = start + batch_size
      x_var = Variable(torch.FloatTensor(x_train[start:end]))
      y_var = Variable(torch.LongTensor(y_train[start:end]))
      optimizer.zero_grad()
      out = model(x_var)
      loss = criterion(out, y_var)
      loss.backward()
      optimizer.step()

      values, labels = torch.max(out, 1) 
      # returns the maximum value and its position
      train_correct = np.sum(labels.data.numpy() == y_train[start:end])
      train_loss += loss.item()*batch_size

    train_loss = train_loss/len(x_train)

    if epoch % 20 == 0:
      print("Epoch: {} \tTrain AVG Loss: {} \tTrain Accuracy: {}".format(epoch+1, train_loss, train_correct / len(y_train[start:end]) ))

Epoch: 1 	Train AVG Loss: 0.5260684958478283 	Train Accuracy: 0.859375
Epoch: 21 	Train AVG Loss: 0.31584047514737223 	Train Accuracy: 0.875
Epoch: 41 	Train AVG Loss: 0.2890207961360395 	Train Accuracy: 0.890625
Epoch: 61 	Train AVG Loss: 0.2572344010110933 	Train Accuracy: 0.84375
Epoch: 81 	Train AVG Loss: 0.2578755312311453 	Train Accuracy: 0.84375
Epoch: 101 	Train AVG Loss: 0.24121378924363063 	Train Accuracy: 0.890625
Epoch: 121 	Train AVG Loss: 0.23852366945973827 	Train Accuracy: 0.921875
Epoch: 141 	Train AVG Loss: 0.24756792036339328 	Train Accuracy: 0.890625
Epoch: 161 	Train AVG Loss: 0.2400628289225179 	Train Accuracy: 0.9375
Epoch: 181 	Train AVG Loss: 0.22835348987188378 	Train Accuracy: 0.84375
Epoch: 201 	Train AVG Loss: 0.2529568417956719 	Train Accuracy: 0.890625
Epoch: 221 	Train AVG Loss: 0.2386459562569226 	Train Accuracy: 0.84375
Epoch: 241 	Train AVG Loss: 0.23367339092015327 	Train Accuracy: 0.921875
Epoch: 261 	Train AVG Loss: 0.23515910866290055 	Train Accur

In [ ]:
x_test = test.iloc[:,:].values

x_test_var = Variable(torch.FloatTensor(x_test), requires_grad=True)
with torch.no_grad():
  test_result = model(x_test_var)
values, labels = torch.max(test_result,1)
Survived = labels.data.numpy()

In [ ]:
# import csv

test_passenger = test_data['PassengerId']

#test_data.head()
submission = [['PassengerId', 'Survived']]

for i in range(len(Survived)):
  submission.append([test_passenger[i], Survived[i]])



In [ ]:
with open('submission.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerows(submission)

# Keras

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam
from keras.utils import np_utils

In [ ]:
from keras.utils import to_categorical

# 要将y转化成one-hot编码，才能给keras训练
y_train = to_categorical(y_train)

In [ ]:
#x_train.shape
len(x_train[0])

21

In [ ]:
y_train.shape

(891, 2)

In [ ]:
if __name__ == '__main__':
    
    model = Sequential()

    model.add(Dense(input_dim = len(x_train[0]),units=50, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 50, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 50, activation = 'relu'))
    model.add(Dense(units = 2, activation='softmax'))
    model.summary()

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    model.fit(x_train, y_train, batch_size = 64, epochs = 30)

    result_train = model.evaluate(x_train, y_train)
    pred_result = model.predict_classes(x_test,verbose=0)

    submission = pd.DataFrame({'PassengerId': test_passenger, 'Survived': pred_result}).to_csv('submission.csv', index=False, header=True)
    print('Training Accuracy:',result_train[1])



Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 50)                1100      
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 102       
Total params: 6,302
Trainable params: 6,302
Non-trainable params: 0
____________________________________________________